In [17]:
import pandas as pd
from sqlalchemy import create_engine, types
import cx_Oracle as oci

In [44]:
PORT_NUMBER = 8099
db_ip = 'localhost'
db_port = '1522'
db_serviceName = 'xe'
db_id = 'DJ2019'
db_pw = 'DJ2019'
db_userTableName = 'BPS_USERS'
db_userTableColumns = ('NAME', 'KEY', 'BALANCE')
db_blockTableName = 'BPS_BLOCK'
db_blockTableColumns = ('BLOCKINDEX', 'PREVIOUSHASH', 'TIMESTAMP', 'DATA', 'CURRENTHASH', 'PROOF')
db_txTableName = 'BPS_TXDATA'
db_txTableColumns = ('COMMIT_YN', 'SENDER', 'AMOUNT', 'RECEIVER', 'UUID', 'tx_Validity')
g_receiveNewBlock = "/node/receiveNewBlock"
g_difficulty = 4
g_maximumTry = 100
g_nodeList = {'trustedServerAddress':'8099'}

In [19]:
def makeUpdateQuery(tableName, setValue, whereCondition):
    print('\tFunction "makeUpdateQuery" executed')
    setValueInput = ''
    for key, value in setValue.items():
        setValueInput += str(key)
        setValueInput += ' = '
        if (isNumber(value)):
            setValueInput += "%s" % str(value)
        else:
            setValueInput += "'%s'" % str(value)
        setValueInput += ', '
    setValueInput = setValueInput.rstrip(', ')

    whereConditionInput = ''
    for key, value in whereCondition.items():
        if (type(value) == list):
            whereConditionInput += '('
            for eachValue in value:
                whereConditionInput += str(key)
                whereConditionInput += ' = '
                if (isNumber(eachValue)):
                    whereConditionInput += str(eachValue)
                else:
                    whereConditionInput += "'%s'" % str(eachValue)
                whereConditionInput += ' OR '
            whereConditionInput = whereConditionInput.rstrip(' OR ')
            whereConditionInput += ')'
        else:
            whereConditionInput += str(key)
            whereConditionInput += ' = '
            if (isNumber(value)):
                whereConditionInput += str(value)
            else:
                whereConditionInput += "'%s'" % str(value)
        whereConditionInput += ' AND '
    whereConditionInput = whereConditionInput.rstrip(' AND ')
    updateQuery = 'UPDATE %s SET %s WHERE %s' % (tableName, setValueInput, whereConditionInput)
    print(updateQuery)
    return updateQuery

In [20]:
def selectTable(tableName, columns, engine, whereCondition = None):
    print('\tFunction "selectTable" executed')   
    
    if (whereCondition != None):
        whereConditionInput = ''
        for key, value in whereCondition.items():
            if (type(value) == list):
                whereConditionInput += '('
                for eachValue in value:
                    whereConditionInput += str(key)
                    whereConditionInput += ' = '
                    if (isNumber(eachValue)):
                        whereConditionInput += str(eachValue)
                    else:
                        whereConditionInput += "'%s'" % str(eachValue)
                    whereConditionInput += ' OR '
                whereConditionInput = whereConditionInput.rstrip(' OR ')
                whereConditionInput += ')'
            else:
                whereConditionInput += str(key)
                whereConditionInput += ' = '
                if (isNumber(value)):
                    whereConditionInput += str(value)
                else:
                    whereConditionInput += "'%s'" % str(value)
            whereConditionInput += ' AND '
        whereConditionInput = whereConditionInput.rstrip(' AND ')
        selectQuery = 'SELECT * FROM %s WHERE %s' % (tableName, whereConditionInput)
    else:
        selectQuery = 'SELECT * FROM %s' % tableName

    try:
        resultData = pd.read_sql_query(selectQuery, engine)
    except:
        print('Table select error, There are no table named "%s" in db. \n It will be created' % tableName)
        createTable(tableName, columns)
        resultData = pd.read_sql_query(selectQuery, engine)
    resultData.rename(columns=lambda x: x.strip().upper(), inplace=True)
    return resultData


In [21]:
def isNumber(s):
    if (type(s) == int or type(s) == float):
        return True
    else:
        return False

In [22]:
def updateTable(tableName, setValue, whereCondition):
    print('\tFunction "updateTable" executed')

    connectComplete = False
    cursorComplete = False

    try:
        connectInfo = db_id + '/' + db_pw + '@' + db_ip + ':' + db_port + '/' + db_serviceName
        oracleConnect = oci.connect(connectInfo)
        connectComplete = True
        oracleCursor = oracleConnect.cursor()
        cursorComplete = True

        updateQuery = makeUpdateQuery(tableName, setValue, whereCondition)
        oracleCursor.execute(updateQuery)
        oracleConnect.commit()
        oracleCursor.close()
        oracleConnect.close

    except:
        if (cursorComplete == True):
            oracleCursor.close()
        if (connectComplete == True):
            oracleConnect.close


In [23]:
def transferInfoCheck(sender, senderUserkey, receiver, receiverUserkey, amount, uuid):

    print('Function "updateTable" executed')
    
    connectInfo = 'oracle+cx_oracle://%s:%s@%s:%s/%s' % (db_id, db_pw, db_ip, db_port, db_serviceName)
    engine = create_engine(connectInfo)
    
    whereCondition = {}
    whereCondition['USERNAME'] = sender
    whereCondition['USERKEY'] = senderUserkey
    senderData = selectTable(db_userTableName, db_userTableColumns, engine, whereCondition)
    if (len(senderData) == 0):
        print("No sender matched")
        return "USER_INFO_NOT_MATCH"
    
    whereCondition = {}
    whereCondition['USERNAME'] = receiver
    whereCondition['USERKEY'] = receiverUserkey
    receiverData = selectTable(db_userTableName, db_userTableColumns, engine, whereCondition)
    if (len(receiverData) == 0):
        print("No receiver matched")
        return "USER_INFO_NOT_MATCH"
    
    resultMessage = moneyTransferCommit(senderData, receiverData, amount, uuid)
    
    return resultMessage

In [24]:
def moneyTransferCommit(senderData, receiverData, amount, uuid):
        
    connectInfo = 'oracle+cx_oracle://%s:%s@%s:%s/%s' % (db_id, db_pw, db_ip, db_port, db_serviceName)
    engine = create_engine(connectInfo)
    
    whereCondition = {}
    whereCondition['UUID'] = uuid
    uuidValidity = selectTable(db_txTableName, db_txTableColumns, engine, whereCondition)
    if (len(uuidValidity) == 0):
        print("No uuid matched")
        return "UUID_NOT_MATCH"
    
    if (uuidValidity['TX_VALIDITY'][0] != '0'):
        print("Data already processed")
        return "ALREADY_PROCESSED"
    
    if (senderData['BALANCE'][0] >= amount): # 보내려는 금액보다 잔고가 크고 개별 트랙잭션 유효성이 null 값일 때 아래 구문 실행.
        setValue = {}
        setValue['BALANCE'] = float(senderData['BALANCE'][0]) - float(amount)
        whereCondition = {}
        whereCondition['USERNAME'] = senderData['USERNAME'][0]
        whereCondition['USERKEY'] = senderData['USERKEY'][0]
        updateTable(db_userTableName, setValue, whereCondition)
        
        setValue = {}
        setValue['BALANCE'] = float(receiverData['BALANCE'][0]) + float(amount)
        whereCondition = {}
        whereCondition['USERNAME'] = receiverData['USERNAME'][0]
        whereCondition['USERKEY'] = receiverData['USERKEY'][0]
        updateTable(db_userTableName, setValue, whereCondition)
        
        setValue = {}
        setValue['TX_VALIDITY'] = '1'
        whereCondition = {}
        whereCondition['uuid'] = uuid
        updateTable(db_txTableName, setValue, whereCondition)
        
        return "SUCCESS"
    else:
        return "LACK_OF_BALANCE"

In [46]:
transferInfoCheck('CC', 'cc', 'BB', 'bb', 3, 2222)

Function "updateTable" executed
	Function "selectTable" executed
	Function "selectTable" executed
	Function "selectTable" executed
Data already processed


'ALREADY_PROCESSED'

In [25]:
def abc(**kwargs):
    result = ""
    for (key, value) in kwargs.items():
        result += str(key)
        result += '='
        result += str(value)
    print(result)

In [13]:
abc(username = 123123)

username=123123


In [45]:
def insertData(tableName, **kwargs):

    connectComplete = False
    cursorComplete = False

    try:
        connectInfo = db_id + '/' + db_pw + '@' + db_ip + ':' + db_port + '/' + db_serviceName
        oracleConnection = oci.connect(connectInfo)
        connectComplete = True
        oracleCursor = oracleConnection.cursor()
        cursorComplete = True

        while (True):
            try:
                keyTemp = str(randint(0, 10000000))  # 키값을 난수로 생성
                key = keyTemp.zfill(10)  # 키 자릿수를 10자리로 맞춰줌
                detailInsert = ''
                for (key, value) in kwargs.items():
                    detailInsert += ':%s, ' % str(key)
                detailInsert = detailInsert.rstrip(', ')
                insertQuery = 'INSERT INTO %s VALUES(%s)' % (tableName, detailInsert)
                oracleCursor.execute(insertQuery, kwargs)
                oracleConnection.commit()
                break
            except:
                continue
        oracleCursor.close()
        oracleConnection.close
    except:
        if (cursorComplete == False):
            oracleCursor.close()
        if (connectComplete == False):
            oracleConnection.close
        print('DB Connection Error')
        return

In [48]:
insertData('BPS_USERS', USERID='123', KEY='2344', BALANCE='333')

In [35]:
from random import *